In [3]:
#pip installations here
!pip install polars
!pip install --upgrade pip

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/amex-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


# initializing variables

In [5]:
# CONSTANTS
MISSINGNESS_THRESHOLD = 0.99
correlation_threshold=0.9
point_biserial_threshold=0.01
stat_sig=0.05

# List of specified categorical columns
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','customer_ID','S_2']

# Reading training data and identifying important columns

In [6]:
# Read the CSV 
df_train = pd.read_csv('train_data.csv')
print(df_train.columns)

['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_63', 'D_64', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'D_66', 'B_20', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'B_30', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_2

In [11]:
#show all data 
# Ensure all rows are printed
"""pd.set_option('display.max_rows', None)

# Create DataFrame with column names and data types
df_dtypes = pd.DataFrame({"Column Name": df_train.columns, "Data Type": df_train.dtypes})

# Print full DataFrame
print(df_dtypes)

# Reset display option if needed
pd.reset_option('display.max_rows')"""

'pd.set_option(\'display.max_rows\', None)\n\n# Create DataFrame with column names and data types\ndf_dtypes = pd.DataFrame({"Column Name": df_train.columns, "Data Type": df_train.dtypes})\n\n# Print full DataFrame\nprint(df_dtypes)\n\n# Reset display option if needed\npd.reset_option(\'display.max_rows\')'

In [9]:
#identify the unique values of the categorical columns
# Loop through each column and print its unique values
for col in categorical_columns:
    if col in df_train.columns:  # Ensure the column exists in df_train
        print(f"Column: {col}")
        print(df_train[col].unique())  # Print unique values
        print("-" * 50)
    else:
        print(f"Warning: Column '{col}' not found in df_train")

Column: B_30
shape: (4,)
Series: 'B_30' [f64]
[
	null
	0.0
	1.0
	2.0
]
--------------------------------------------------
Column: B_38
shape: (8,)
Series: 'B_38' [f64]
[
	null
	1.0
	2.0
	3.0
	4.0
	5.0
	6.0
	7.0
]
--------------------------------------------------
Column: D_114
shape: (3,)
Series: 'D_114' [f64]
[
	null
	0.0
	1.0
]
--------------------------------------------------
Column: D_116
shape: (3,)
Series: 'D_116' [f64]
[
	null
	0.0
	1.0
]
--------------------------------------------------
Column: D_117
shape: (8,)
Series: 'D_117' [f64]
[
	null
	-1.0
	1.0
	2.0
	3.0
	4.0
	5.0
	6.0
]
--------------------------------------------------
Column: D_120
shape: (3,)
Series: 'D_120' [f64]
[
	null
	0.0
	1.0
]
--------------------------------------------------
Column: D_126
shape: (4,)
Series: 'D_126' [f64]
[
	null
	-1.0
	0.0
	1.0
]
--------------------------------------------------
Column: D_63
shape: (6,)
Series: 'D_63' [str]
[
	"XM"
	"CL"
	"XL"
	"CO"
	"XZ"
	"CR"
]
-----------------------

NOTE:

train_data.csv Data types are mainly in float64 and string 

customerID: string 

S-2(date): string
( we can see that the train statement dates ranges from "2017-12-23" to "2018-03-14" this show )

categorical data colums:
['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
note: D_63,D_64 : string




In [ ]:
# Convert 'S_2' to datetime format
df_train["S_2"] = pd.to_datetime(df_train["S_2"], format="%Y-%m-%d")

# 1. Count UNIQUE statements per customer
unique_counts = (
    df_train.drop_duplicates(subset=["customer_ID", "S_2"])  # Remove duplicate dates per customer
    .groupby("customer_ID")
    .size()
    .reset_index(name="statement_count")
    .sort_values("statement_count", ascending=False)
)

# 2. Create pie chart
plt.figure(figsize=(10, 8))
colors = plt.cm.tab20.colors

plt.pie(
    unique_counts["statement_count"],
    labels=unique_counts["customer_ID"],
    autopct=lambda p: f"{p:.1f}%\n({int(p*sum(unique_counts['statement_count'])/100)})",
    startangle=90,
    colors=colors,
    wedgeprops={"edgecolor": "white", "linewidth": 1}
)

plt.title("Unique Bank Statements Distribution by Customer\n(Polars + Matplotlib)", pad=20)
plt.axis("equal")

# 3. Add table with exact counts
plt.table(
    cellText=unique_counts.values,
    colLabels=unique_counts.columns,
    loc="bottom",
    bbox=[0, -0.4, 1, 0.3]
)

plt.tight_layout()
plt.show()

# Print the raw data
print("Unique statement counts per customer:")
print(unique_counts)

# Missing values removal functions

In [ ]:
#gets the colums with 99% missing data
def missing_columns(data):
    missing_percentages = data.isnull().mean()
    missing_columns = missing_percentages[missing_percentages > MISSINGNESS_THRESHOLD].index
    return missing_columns
    
#removes colums with 99% of data missing ^takes colums from above function
def drop_columns_with_many_missing_values_in_derived_dataset(data,base_columns_to_drop):
  columns_to_drop=[]
  for i in range(0,len(base_columns_to_drop)):
    columns_to_drop.append(base_columns_to_drop[i]+'_')
  columns_to_drop = [col for col in data.columns if (any(word in col for word in columns_to_drop) and ("last" not in col))]
  data_filtered = data.drop(columns=columns_to_drop, errors='ignore')
  return data_filtered

# Handling missing colums

In [ ]:
#show the bad cols
list_of_bad_col = missing_columns(df_train)
list_of_bad_col

In [ ]:
# removing the bad colums
df_train = drop_columns_with_many_missing_values_in_derived_dataset(df_train,list_of_bad_col)
df_train.shape

# aggregating new data

In [ ]:
def enhanced_feature_engineering(df_train):
    """
    Performs comprehensive feature engineering on financial data.
    
    Args:
        df: Input DataFrame containing financial features
        
    Returns:
        DataFrame with original and engineered features
    """
    # Data Cleaning
    df_train = df_train.replace([np.inf, -np.inf], np.nan)
    df_train = df_train.fillna(-127)  # Consider using a more meaningful null placeholder
    
    # Feature Identification
    non_feature_cols = ['customer_ID', 'S_2', 'target']
    num_features = [col for col in df_train.columns if col not in non_feature_cols]
    
    
    # Numerical Transformations (now applies to all numerical features)
    num_features = [col for col in num_features if col not in categorical_columns]
    for col in num_features:
        if df_train[col].dtype in ['float64', 'int64']:
            # Skip log transform if negative values exist
            if (df_train[col] >= 0).all():
                df_train[f'{col}_log'] = np.log1p(df_train[col])
    
    # Feature Groupings (now includes all features, not just *_last)
    P_columns = [col for col in df_train.columns if col.startswith('P_')]
    S_columns = [col for col in df_train.columns if col.startswith('S_')]
    B_columns = [col for col in df_train.columns if col.startswith('B_')]
    D_columns = [col for col in df_train.columns if col.startswith('D_')]
    R_columns = [col for col in df_train.columns if col.startswith('R_')]
    
    # 6. Aggregate Features with safer calculations
    df_train['total_payments'] = df_train[P_columns].sum(axis=1)
    df_train['total_spending'] = df_train[S_columns].sum(axis=1)
    df_train['total_balance'] = df_train[B_columns].sum(axis=1)
    df_train['total_delinquency'] = df_train[D_columns].sum(axis=1)
    df_train['total_risk'] = df_train[R_columns].sum(axis=1)
    
    # 7. Ratio Features with zero-division protection
    epsilon = 1e-6
    df_train['risk_to_payment_ratio'] = np.where(
        df_train['total_payments'].abs() > epsilon,
        df_train['total_risk'] / (df_train['total_payments'] + epsilon),
        np.nan
    )
    df_train['delinquency_to_balance_ratio'] = np.where(
        df_train['total_balance'].abs() > epsilon,
        df_train['total_delinquency'] / (df_train['total_balance'] + epsilon),
        np.nan
    )
    
    # 8. Memory Optimization
    for col in df_train.select_dtypes(['float64']).columns:
        if col != "customer_ID":
            df_train[col] = df_train[col].astype(np.float32)
            
    for col in df_train.select_dtypes(['int64']).columns:
        if col != "customer_ID":
            df_train[col] = df_train[col].astype(np.int32)
    
    print(f'Shape after feature engineering: {df_train.shape}')
    print(f'Number of new features created: {len(df_train.columns) - len(non_feature_cols)}')
    
    return df_train

In [ ]:
df_train = enhanced_feature_engineering(df_train)

In [ ]:
missing_columns_derived=missing_columns(df_train)

In [ ]:
print(missing_columns_derived)

# Reducing number of features using correlation

# *handling low correlation data*

In [ ]:

from scipy.stats import pointbiserialr

def find_low_point_biserial_correlations_columns(df, target_column, category, suffix):

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(-127, inplace=True)


    columns_to_see = [col for col in df.columns if (col.startswith(category) and col.endswith(suffix)) or col == target_column]
    df = df[columns_to_see]

    correlation_results = []


    for col in columns_to_see:
        if col != target_column:
            correlation, p_value = pointbiserialr(df[target_column], df[col])
            correlation_results.append((col, correlation, p_value))


    correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Point-Biserial Correlation', 'p-value'])


    filtered_df = correlation_df[(correlation_df['Point-Biserial Correlation'].abs() < point_biserial_threshold) & (correlation_df['p-value'] < stat_sig)]


    return filtered_df['Feature'].tolist()

In [ ]:

category= ["B","P","S","D","R"]
suffix=['first', 'mean', 'std', 'min', 'max']

In [ ]:

def remove_low_correlation_columns(df):
    all_cols_to_omit=[]
    for c in category:
        for s in suffix:
            columns_to_omit=find_low_point_biserial_correlations_columns(df,"target",c,s)
            all_cols_to_omit.append(columns_to_omit)
    return all_cols_to_omit

In [ ]:

all_low_correlation_columns=remove_low_correlation_columns(df_train)

In [ ]:

for l in all_low_correlation_columns:
  df_train.drop(columns=l, errors='ignore',inplace=True)

# *handling high correlated data*

In [ ]:
from scipy.stats import pearsonr

def find_highly_correlated_columns(df, category, suffix):
    # df=df.to_pandas()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(-127, inplace=True)


    numerical_columns = [col for col in df.columns if col.startswith(category) and col.endswith(suffix)]
    df = df[numerical_columns]


    correlation_matrix = df.corr(method='pearson').abs()


    correlated_pairs = set()
    columns_to_remove = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            if correlation_matrix.iloc[i, j] > correlation_threshold:
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]


                correlated_pairs.add((col1, col2))

                columns_to_remove.add(col2)

    return list(columns_to_remove)

def remove_high_correlation_columns(df):
    all_cols_to_omit=[]
    for c in category:
        for s in suffix:
            columns_to_omit=find_highly_correlated_columns(df,c,s)
            all_cols_to_omit.append(columns_to_omit)
    return all_cols_to_omit

In [ ]:

all_high_correlation_columns=remove_high_correlation_columns(df_train)

In [ ]:

for l in all_high_correlation_columns:
  df_train.drop(columns=l, errors='ignore',inplace=True)

test

In [ ]:
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','customer_ID','S_2','target']

In [ ]:

df_sampled = df_train.sample(n=20000, seed=42)

df_numeric = df_sampled.drop(cat_features)

correlation_matrix = df_numeric.corr()

print(correlation_matrix)

print(type(correlation_matrix))

# Removing Outliers

In [ ]:

train_data = df_train

In [ ]:
train_data_cat = train_data[cat_features]

In [ ]:
train_data_num = train_data.drop(cat_features)

In [ ]:
for col in train_data_num.columns:
    if col not in cat_features:
        
        Q1 = train_data_num.select(pl.col(col).quantile(0.25)).to_numpy()[0][0]
        Q3 = train_data_num.select(pl.col(col).quantile(0.75)).to_numpy()[0][0]
        IQR = Q3 - Q1
        
        # Calculate lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Filter the DataFrame to keep only rows within bounds
        train_data_num = train_data_num.filter(pl.col(col).is_between(lower_bound, upper_bound))

# After the loop, train_data will contain only rows without outliers in any specified column
print(train_data_num)

In [ ]:
# 4. Standardize the numerical columns (mean=0, std=1)
train_data_num = (train_data_num - train_data_num.mean()) / train_data_num.std()

In [ ]:
train_data_num

In [ ]:
# 5. Concatenate the numerical and categorical data
train_data = pd.concat([train_data_cat, train_data_num], axis=1)

In [ ]:
train_data.head()

# Handling Categorical Variables 

In [ ]:
# train_data_cat.null_count()
# print(500/len(train_data))

In [ ]:
train_data = train_data.dropna()

In [ ]:
train_data['target'].value_counts()

In [ ]:
train_data_cp = train_data

In [ ]:
df_majority = df_train[df_train['target'] == 0]
df_minority = df_train[df_train['target'] == 1]

# Determine the number of samples for upsampling
n_samples = len(df_majority)

# Upsample minority class
df_minority_upsampled = df_minority.sample(
    n=n_samples, 
    replace=True,  # Replace to allow upsampling
    random_state=42  # Set the seed for reproducibility
)
# Combine majority and upsampled minority classes
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

print(df_upsampled)


In [ ]:
train_data = df_upsampled

In [ ]:
train_data['target'].value_counts()

In [ ]:
train_data_cat_encoded = pd.get_dummies(train_data_cat)
print(train_data_cat_encoded)